In [40]:
import pandas as pd
import numpy as np
import cv2
import os
from alive_progress import alive_bar

In [41]:
rows = []
class_size = 5
n_classes = int(255/class_size)
for i in range(n_classes):
    for j in range(n_classes):
        for k in range(n_classes):
            rows.append([i * class_size, j * class_size, k * class_size, 0, 'none'])

df = pd.DataFrame(rows, columns=['r', 'g', 'b', 'n_files', 'files'])

df.head()

,r,g,b,n_files,files
0,0,0,0,0,none
1,0,0,5,0,none
2,0,0,10,0,none
3,0,0,15,0,none
4,0,0,20,0,none


In [42]:
files = os.listdir('resized_pics/')
with alive_bar(len(files)) as bar:
    for f in files:
        img = cv2.imread(f'resized_pics/{f}')
        avg_color_per_row = np.average(img, axis=0)
        avg_color = np.average(avg_color_per_row, axis=0)

        r, g, b = [x - x % class_size if x != 255 else 250 for x in avg_color]
        index = (r / class_size) * (n_classes ** 2) + (g / class_size) * n_classes + (b / class_size)
        index = int(index)

        files = df.loc[index, 'files']
        files = files.split(';') if files != 'none' else []
        files.append(f)
        df.at[index, 'files'] = ';'.join(files)
        df.at[index, 'n_files'] = len(files)

        bar()

|████████████████████████████████████████| 5605/5605 [100%] in 1.2s (4570.02/s)


In [45]:
df.loc[df['files'] != 'none'][:10]

,r,g,b,n_files,files
5320,10,10,80,1,ru2oym.jpg
5360,10,15,25,1,rqukub.jpg
5373,10,15,90,1,rsi5zk.jpg
7960,15,15,20,2,ru4nvq.jpg;rtc1w2.jpg
7970,15,15,70,1,rqrjxn.jpg
8011,15,20,20,2,rnwsal.jpg;rm1s3y.jpg
8066,15,25,40,3,ru0hqt.jpg;rs5xga.jpg;ru0i0i.jpg
8118,15,30,45,1,rttti6.jpg
8224,15,40,65,1,rqzgn3.jpg
9936,15,205,210,1,rka1sb.jpg


In [44]:
df.to_csv('classification.csv', encoding='utf-8')